# GPT-3.5 TURBO model prototype
In dit notebook, zullen we testen of we met het GPT-3.5-turbo model van OpenAI proberen om een chatbot te maken voor iYYU. Deze zal uiteindelijk vragen moeten kunnen beantwoorden over de privacy en legal statements van iYYU, maar ook over de werking van de app en het instellen van de appinstellingen.

We gebruiken publieke data van iYYU om de chatbot te trainen. Deze data is te vinden in de map `text`:
- `legal.txt` bevat de legal statements van iYYU, afkomstig van de website van iYYU.
- `privacy.txt` bevat de privacy statements van iYYU, afkomstig van de website van iYYU.
- `account-settings.json` bevat een JSON-formatted lijst van gecategoriseerde account instellingen van iYYU, met alle opties, mogelijke waarden, de geadviseerde waarde en een korte beschrijving van de instelling.

In [24]:
def remove_newlines(serie):
    serie = serie.str.replace('\n', ' ')
    serie = serie.str.replace('\\n', ' ')
    serie = serie.str.replace('  ', ' ')
    return serie

## Bestanden inladen
Hieronder laden we alle bestanden in die we nodig hebben om de chatbot te trainen. We gebruiken de `legal.txt` en `privacy.txt` bestanden om de chatbot te trainen op de legal en privacy statements van iYYU. We gebruiken het `account-settings.json` bestand om de chatbot te trainen op de account instellingen van iYYU. We gebruiken de `questions.txt` bestand om de chatbot te trainen op vragen die gebruikers kunnen stellen over de app en de account instellingen.

Deze worden automatisch ingeladen vanuit de `text` map.

We lezen hier alle bestanden uit en slaan ze op in een `.csv` bestand nadat ervoor hebben gezorgd dat alle linebreaks eruit zijn gehaald. Dit `.csv` bestand wordt opgeslagen in de `processed` map als `scraped.csv`.

In [14]:
import os
import pandas as pd

# Create a list to store the text files
texts = []

# Get all the text files in the text directory
for file in os.listdir("text/"):
    # Open the file and read the text
    with open("text/" + file, "r") as f:
        text = f.read()

        texts.append((file, text))

# Create a dataframe from the list of texts
df = pd.DataFrame(texts, columns=['fname', 'text'])

# Set the text column to be the raw text with the newlines removed
df['text'] = remove_newlines(df.text)
df.to_csv('processed/scraped.csv')
df.head()

C:\Users\gaagj\AppData\Local\Temp\ipykernel_85400\2966875972.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  serie = serie.str.replace('\\n', ' ')


,fname,text
0,legal.txt,iYYU Terms & Conditions These Terms and Condit...
1,privacy.txt,iYYU Privacy Policy This Privacy Policy sets o...
2,training-data.json,"{ ""visibility"": { ""description"": ""By choosi..."


## Data preprocessing

We willen de data preprocessen zodat we deze kunnen gebruiken om de chatbot te trainen. Het is hiervoor nodig om de data te tokenizen. We gebruiken hiervoor de `tiktoken` library. Deze library is ontworpen om te werken met de GPT-3 modellen van OpenAI. We gebruiken de `cl100k_base` tokenizer die is ontworpen om te werken met de `ada-002` model.

In [25]:
import tiktoken

# De cl100k_base tokenizer inladen, die is ontworpen om te werken met het ada-002 model.
tokenizer = tiktoken.get_encoding("cl100k_base")

df = pd.read_csv('processed/scraped.csv', index_col=0)
df.columns = ['title', 'text']

# Een nieuwe kolom genaamd 'n_tokens' toevoegen aan de dataframe, die de lengte van de tokenized text bevat.
df['n_tokens'] = df.text.apply(lambda x: len(tokenizer.encode(x)))

df.head()

,title,text,n_tokens
0,legal.txt,iYYU Terms & Conditions These Terms and Condit...,3131
1,privacy.txt,iYYU Privacy Policy This Privacy Policy sets o...,2498
2,training-data.json,"{ ""visibility"": { ""description"": ""By choosi...",950


### Chunking

We willen de data chunken zodat het GPT-3.5-turbo model deze kan gebruiken. Er zit namelijk een limiet aan het aantal tokens dat het model in één keer kan verwerken.

De code hieronder zal de tekst die we eerder hebben verwerkt naar het `scraped.csv` bestand in chunks van maximaal 500 tokens opsplitsen. We zullen dit doen door te splitsen om zinnen (.) en deze net zo lang toe voegen aan een chunk totdat deze de limiet van 500 tokens bereikt. We slaan deze chunks op in een nieuw .csv bestand genaamt `embeddings.csv`.

In [16]:
max_tokens = 500

# Function to split the text into chunks of a maximum number of tokens
def split_into_many(text, max_tokens=max_tokens):
    # Split the text into sentences
    sentences = text.split('. ')

    # Get the number of tokens for each sentence
    n_tokens = [len(tokenizer.encode(" " + sentence)) for sentence in sentences]

    chunks = []
    tokens_so_far = 0
    chunk = []

    # Loop through the sentences and tokens joined together in a tuple
    for sentence, token in zip(sentences, n_tokens):

        # If the number of tokens so far plus the number of tokens in the current sentence is greater
        # than the max number of tokens, then add the chunk to the list of chunks and reset
        # the chunk and tokens so far
        if tokens_so_far + token > max_tokens:
            chunks.append(". ".join(chunk) + ".")
            chunk = []
            tokens_so_far = 0

        # If the number of tokens in the current sentence is greater than the max number of
        # tokens, go to the next sentence
        if token > max_tokens:
            continue

        # Otherwise, add the sentence to the chunk and add the number of tokens to the total
        chunk.append(sentence)
        tokens_so_far += token + 1

    return chunks

shortened = []

# Loop through the dataframe
for row in df.iterrows():

    # If the text is None, go to the next row
    if row[1]['text'] is None:
        continue

    # If the number of tokens is greater than the max number of tokens, split the text into chunks
    if row[1]['n_tokens'] > max_tokens:
        shortened += split_into_many(row[1]['text'])

    # Otherwise, add the text to the list of shortened texts
    else:
        shortened.append(row[1]['text'])

In [26]:
df = pd.DataFrame(shortened, columns=['text'])
df['n_tokens'] = df.text.apply(lambda x: len(tokenizer.encode(x)))

df.head()

,text,n_tokens
0,iYYU Terms & Conditions These Terms and Condit...,480
1,By accessing or using (any part of) the Platfo...,478
2,You can become a Space Member by agreeing to t...,441
3,"To the maximum extent permitted by law, iYYU h...",452
4,"Are not or do not contain spam, are not machin...",492


## Embeddings

We zullen nu de embeddings van de chunks berekenen. Dit zijn vectors die de betekenis van de tekst weergeven. We zullen hiervoor het openai model `ada-002` gebruiken. Deze vectors worden bepaalt door de context van de tekst. Dit betekent dat de vector van een zin afhankelijk is van de zinnen die ervoor en erna komen. Dit maakt het mogelijk voor het model om relaties tussen woorden en zinnen te leren.
De berekende embeddings zullen we toevoegen aan de dataset.

In [18]:
import openai
# pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

openai.api_key = os.getenv('OPENAI_API_KEY')

df['embeddings'] = df.text.apply(lambda x: openai.Embedding.create(input=x, engine='text-embedding-ada-002')['data'][0]['embedding'])
df.to_csv('processed/embeddings.csv')
df.head()

,text,n_tokens,embeddings
0,iYYU Terms & Conditions These Terms and Condit...,480,"[-0.00584253529086709, -0.015280477702617645, ..."
1,By accessing or using (any part of) the Platfo...,478,"[0.015693487599492073, -0.01975192129611969, 0..."
2,You can become a Space Member by agreeing to t...,441,"[0.005822580307722092, -0.028022408485412598, ..."
3,"To the maximum extent permitted by law, iYYU h...",452,"[-0.001226945430971682, -0.013233720324933529,..."
4,"Are not or do not contain spam, are not machin...",492,"[-0.009652386419475079, -0.010911393910646439,..."


## Context berekenen

We zullen nu de context berekenen voor de input van de gebruiker. Dit doen we door de embeddings van de input te vergelijken met de embeddings van de chunks. We zullen de chunks sorteren op afstand van de input en deze toevoegen aan de context totdat de context de limiet van tokens is bereikt.

In [21]:
from openai.embeddings_utils import distances_from_embeddings

def create_context(
        question, df, max_len=1800, size="ada"
):
    """
    Create a context for a question by finding the most similar context from the dataframe
    """

    # Get the embeddings for the question
    q_embeddings = openai.Embedding.create(input=question, engine='text-embedding-ada-002')['data'][0]['embedding']

    # Get the distances from the embeddings
    df['distances'] = distances_from_embeddings(q_embeddings, df['embeddings'].values, distance_metric='cosine')

    returns = []
    cur_len = 0

    # Sort by distance and add the text to the context until the context is too long
    for i, row in df.sort_values('distances', ascending=True).iterrows():

        # Add the length of the text to the current length
        cur_len += row['n_tokens'] + 4

        # If the context is too long, break
        if cur_len > max_len:
            break

        # Else add it to the text that is being returned
        returns.append(row["text"])

    # Return the context
    return "\n\n###\n\n".join(returns)

# Model aanroepen

We zullen nu de functie `answer_question` aanroepen. Deze functie zal de context berekenen, afgeleid van de input van de gebruiker, en deze gebruiken om een antwoord te geven op de vraag van de gebruiker.

We geven het model een start prompt mee om aan te geven wat de rol is van het model en hoe het antwoord moet geven. Het is hierbij ook belangrijk dat we het model goed afschermen, zodat het geen antwoord kan geven op onrelevante vragen die buiten de context vallen.

Hiervoor gebruiken we `Prompt Engineering`. Dit is het proces waarbij we een prompt schrijven die een language model een specifieke taak laat uitvoeren. Dit gebeurd aan de hand van een lijst met instructies die het model moet volgen. Een goede prompt verhoogt de kwaliteit en accuraatheid van de output van het model.

In [ ]:
def answer_question(
        df,
        model="gpt-3.5-turbo",
        question="",
        max_len=1800,
        size="ada",
        debug=False,
        max_tokens=150,
        stop_sequence=None
):
    """
    Answer a question based on the most similar context from the dataframe texts
    """
    context = create_context(
        question,
        df,
        max_len=max_len,
        size=size,
    )
    # If debug, print the raw model response
    if debug:
        print("Context:\n" + context)
        print("\n\n")

    try:
        # Create a completions using the question and context
        response = openai.ChatCompletion.create(
            messages=[
                {"role": "system",
                 "content": f"Answer the question based on the context below, and if the question can't be answered based on the context, say \"I don't know. You are the spokesperson of the company iYYU so you will refer to the company by its full name or 'we'. Context: {context}"},
                {"role": "user",
                 "content": question},
            ],
            temperature=0,
            max_tokens=max_tokens,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=stop_sequence,
            model=model,
        )
        return response["choices"][0]['message']['content'].strip()
    except Exception as e:
        print(e)
        return ""

In [22]:
answer_question(df, question="Wat is iYYU? Antwoord in het Nederlands.")

'iYYU is een platform dat gebruikers in staat stelt om in contact te blijven met anderen en berichten te verzenden en ontvangen. Het is eigendom van en wordt beheerd door iYYU B.V., gevestigd in Amsterdam, Nederland.'

In [23]:
answer_question(df, question="Could you give me a small summary of the privacy statement? Explain all the important points as if I were a 5 year old.")

'This is a privacy policy for a website called iYYU. It tells you what information they collect from you when you use their website and how they use it. They promise to keep your information safe and not share it with anyone unless they have to. They also give you some rights, like the right to ask them to delete your information or to see what information they have about you. If you have any questions or concerns, you can contact them.'